# Pandemia dżumdżumy

Rozważmy sobie hipotetyczną kolejną straszną chorobę, która nawiedza nasze społeczeństwo. Czy z wykorzystaniem Pythona możemy sobie zamodelować jej rozprzestrzenianie.

Zadania o charakterze symulacji są jednymi z najlepszych technik prezentowania aspektów programowania obiektowego. Podobnie jak ... gry komputerowe. Dzięki zbliżeniu kodu do metaforycznych tworów - można dostrzec istotny potencjał w tym sposobie organizacji kodu. Poza tym pozwala on również na wcielenie się w rolę lokalnego boga i definiowanie wszystkich praw i możliwości jakie obowiązują w danym miejscu.



_Z kraju ukrytego w wysokich himalajach Dżumurii, jeden z mieszkańców dostał się do starożytnej jaskini, zamarzniętej pod lodowcem setki tysięcy lat temu i zakaził się antycznym - rozmrożonym wirusem. Podrużując do Indii po pomoc medyczną rozpoczął nową światową pandemię choroby znanej jak Dżumdżuma._



![image](https://i.ytimg.com/vi/-1qju6V1jLM/maxresdefault.jpg)

Poznajmy naszych aktorów.

Pierwszym jest oczywiście nasz staruszek świat który musimy zostać jakoś ujęty 

In [60]:
from random import random, sample, randrange

class Swiat(object):
    
    def __init__(self, liczba_ludzi = 0, liczba_chorych = 0, liczba_spotkan = 2):
        self.rok = 2020
        self.dzien = 0
        self.liczba_spotkan = liczba_spotkan
        self.populacji = []
        for _ in range(liczba_chorych):
            nowy_czlowiek = Czlowiek()
            nowa_choroba = Choroba() #tworzymy chorobe
            nowy_czlowiek.choroba = nowa_choroba #przypisujemy chorobe do czlowieka
            self.populacji.append(nowy_czlowiek) #dolacz go do populacji swiata
        for _ in range(liczba_ludzi - liczba_chorych):
            nowy_czlowiek = Czlowiek() #stworz nowego czlowieka
            self.populacji.append(nowy_czlowiek) #dolacz go do populacji swiata
    
    def mija_dzien(self):
        self.dzien += 1
        if self.dzien % 365 == 0:
            self.rok += 1
            self.dzien = 0
        
        for czlowiek in self.populacji:
            #losujemy ludzi z ktorymi sie on tego dnia spotka
            for spotkanego_czlowieka in sample(self.populacji, self.liczba_spotkan):
                czlowiek.kontakt(spotkanego_czlowieka)
                spotkanego_czlowieka.kontakt(czlowiek)
        
        print(f'\nMamy dzien {self.dzien:3} roku {self.rok}', end=' ')
        choroby = []
        
        zywi = []
        for czlowiek in self.populacji:
            if czlowiek.czy_zywy():
                zywi.append(czlowiek)
        self.populacji = zywi         
        
        for czlowiek in self.populacji:
            czlowiek.mija_dzien()
            if czlowiek.choroba is not None:
                choroby.append(czlowiek.choroba)
        for choroba in choroby:
            choroba.mija_dzien()
        self.wypisz_stan_pandemii()
            
    
    def wypisz_stan_pandemii(self):
        liczba_ludzi = len(self.populacji)
        chory = []
        for czlowiek in self.populacji:
            if czlowiek.czy_chory():
                chory.append(czlowiek)
        liczba_chorych = len(chory)
        
        print(f'Populacja liczy {liczba_ludzi:5} osób z których chorych jest {liczba_chorych:5}', end = ' ')
        print(f'Stanowi to {int(liczba_chorych/liczba_ludzi * 100):3}% całej populacji', end =' ')
        
    def uruchom_przez(self, dni):
        for _ in range(dni):
            self.mija_dzien()

Kolejnym jest nasz człowieczek (a w zasadzie ich mnogość)

In [64]:
class Czlowiek(object):
    
    def __init__(self):
        self.zdrowie = 100
        self.odpornosc = 0
        self.choroba = None
    
    def mija_dzien(self):
        
        if self.choroba is not None and self.choroba.czy_koniec(): # chorowal, ale choroba juz mija
            self.choroba = None #wyzdrowial
            self.odpornosc = 100
            
        if self.choroba is None and self.zdrowie < 100:
            self.zdrowie += 2
        elif self.choroba is not None:
            self.zdrowie -= self.choroba.szkodliwosc
        
        if self.odpornosc > 0:
            self.odpornosc -= 1

    def czy_chory(self):
        return self.choroba is not None
    
    def czy_zywy(self):
        return self.zdrowie > 0
    
    def kontakt(self, inny_czlowiek):
        if self.choroba is not None and inny_czlowiek.choroba is None:
            losowa = random() #(0,1)
            if int(losowa * 100) > inny_czlowiek.odpornosc:
                nowa_choroba = Choroba() # tworzona jest kolejna choroba
                inny_czlowiek.choroba = nowa_choroba
            


Ostatnim (przynajmniej na początku) aktorem jest oczywiście choroba, która za pomocą ludzi się roznosi 

In [65]:
class Choroba(object):
    
    def __init__(self):
        self.trwania = 0
        self.szkodliwosc = randrange(5,12) # wylosowac z przedzialu [5,12]
    
    def mija_dzien(self):
        self.trwania += 1
        
    def czy_koniec(self):
        czas_trwania = 15
        return self.trwania > czas_trwania
    

Poza tym potrzebny nam setup (konfiguracja początkowa) oraz start całej symulacji

In [66]:
swiat = Swiat(liczba_ludzi = 10000, liczba_chorych = 1, liczba_spotkan = 5)
swiat.uruchom_przez(180)


Mamy dzien   1 roku 2020 Populacja liczy 10000 osób z których chorych jest  9805 Stanowi to  98% całej populacji 
Mamy dzien   2 roku 2020 Populacja liczy 10000 osób z których chorych jest 10000 Stanowi to 100% całej populacji 
Mamy dzien   3 roku 2020 Populacja liczy 10000 osób z których chorych jest 10000 Stanowi to 100% całej populacji 
Mamy dzien   4 roku 2020 Populacja liczy 10000 osób z których chorych jest 10000 Stanowi to 100% całej populacji 
Mamy dzien   5 roku 2020 Populacja liczy 10000 osób z których chorych jest 10000 Stanowi to 100% całej populacji 
Mamy dzien   6 roku 2020 Populacja liczy 10000 osób z których chorych jest 10000 Stanowi to 100% całej populacji 
Mamy dzien   7 roku 2020 Populacja liczy 10000 osób z których chorych jest 10000 Stanowi to 100% całej populacji 
Mamy dzien   8 roku 2020 Populacja liczy 10000 osób z których chorych jest 10000 Stanowi to 100% całej populacji 
Mamy dzien   9 roku 2020 Populacja liczy 10000 osób z których chorych jest 10000 Stanow

Mamy dzien  82 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien  83 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien  84 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien  85 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien  86 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien  87 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien  88 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien  89 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien  90 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi

Mamy dzien 159 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien 160 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien 161 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien 162 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien 163 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien 164 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien 165 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien 166 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi to   0% całej populacji 
Mamy dzien 167 roku 2020 Populacja liczy  2783 osób z których chorych jest     0 Stanowi

# Jak rozwinąć model dżumdżumy

Przygotowany tu kod można łatwo rozszerzyć - dodając mu pewne wymagania

W szczególności możemy np.

* Dopisać jak wyglądają wzorce zależności - czyli np. pogrupować ludzi względem rodzin, współpracowników;
* Możemy rozszerzyć klasę Człowieka i stworzyć klasę pochodną Lekarz i nadać jej zadania oraz wpływ na kształt rozwoju choroby;
* Wprowadzić szczepienia;
* Wprowadzić kwarantanny, obostrzenia i co nam się spodoba;
* Wprowadzić stadia choroby: inkubacja i postać pełnoobjawowa;
* Wprowadzić różny przebieg w zależności od wieku;
* ...
* Można dodać do człowieka pole płeć i nadać dżumdżumie własności zupełnie innej rodziny chorób;

# Co na teraz 

Pobrać sobie plik `pandemia_dzumdzumy.py` z repozytorium `https://github.com/centrum-mistrzostwa-informatycznego/cmi-edition3` 

* (Dla bardzo początkujących) pytać - ja odpowiadam i wyjaśniam.
* (Dla przeciętnych) uruchomić sobie samodzielnie tę symulację. Może zmodyfikować pewne parametry symulacji. Jak ilość kontaktów, czas trwania choroby. Zmienić parametry życiowe ludzi lub chorób.
* Spróbować wprowadzić jakaś niewielką zmianę w sposobie działania tego świata i zobaczyć jej wpływ na stan świata.